In [ ]:
import sys
import wandb
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
sns.set_palette(sns.color_palette("Paired"))
matplotlib.rcParams.update({'font.size': 10})

fig, axs = plt.subplots(1, 2, figsize=(12,3))
locs = np.arange(4)

heights = [0.2, 0.3, 0.95, 1.]

bar1 = axs[0].bar(locs[:-1], heights[:-1], width=0.4, color="w")
bar1 += axs[0].bar(locs[-1:], heights[-1:], width=0.4, hatch="/", color="w")
labels = ["+ ODE Bias", "+ Second-Order\nBias", "+ Symplectic\nBias", "Hamiltonian\nNN"]
for idx, (rect, label) in enumerate(zip(bar1, labels)):
    height = rect.get_height()
    if idx == (len(labels) - 1):
        axs[0].text(rect.get_x() + rect.get_width() / 2.0, height + 0.09, label, ha='center', va='bottom',
                    bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5', linewidth=1))       
    else:
        axs[0].text(rect.get_x() + rect.get_width() / 2.0, height + 0.07, label, ha='center', va='bottom'

axs[0].bar(locs[3:], heights[3], width=0.4) 
axs[0].bar(locs[2:], heights[2], width=0.4)
axs[0].bar(locs[1:], heights[1], width=0.4) 
axs[0].bar(locs, heights[0], width=0.4)
axs[0].bar(locs, heights, width=0.4, edgecolor='black', fill=False) 
    
axs[0].set_title("Expectation", fontsize=20, pad=40)
axs[0].set_yticks(np.linspace(0, 1, 5))
axs[0].set_yticklabels(["", "25%", "50%", "75%", "100%"], fontsize=14)
axs[0].set_ylabel("Performance", fontsize=16)#, rotation=45)
axs[0].tick_params(axis=u'both', which=u'both',length=0)

axs[0].set_xticks([])
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

heights = [0.2, 0.9, 0.95, 1.0]

bar2 = axs[1].bar(locs[:-1], heights[:-1], width=0.4, color="w", edgecolor='black') 
bar2 += axs[1].bar(locs[-1:], heights[-1:], width=0.4, hatch="/", color="w", edgecolor='black')
# labels = ["+ ODE Bias", "+ Second-Order\nBias", "+ Symplectic\nBias", "+ Simple Function\nBias", "HNN"]
labels = ["+ ODE Bias", "+ Second-Order\nBias", "+ Symplectic\nBias", "Hamiltonian\nNN"]
for idx, (rect, label) in enumerate(zip(bar2, labels)):
    height = rect.get_height()
    if idx == (len(labels) - 1):
        axs[1].text(rect.get_x() + rect.get_width() / 2.0, height + 0.09, label, ha='center', va='bottom',
                    bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5', linewidth=1))       
    else:
        axs[1].text(rect.get_x() + rect.get_width() / 2.0, height + 0.07, label, ha='center', va='bottom')

axs[1].bar(locs[3:], heights[3], width=0.4)
axs[1].bar(locs[2:], heights[2], width=0.4)
axs[1].bar(locs[1:], heights[1], width=0.4)
axs[1].bar(locs, heights[0], width=0.4)
axs[1].bar(locs, heights, width=0.4, edgecolor='black', fill=False) 
    
axs[1].set_title("Reality", fontsize=20, pad=40)
axs[1].set_yticks(np.linspace(0, 1, 5))
axs[1].set_yticklabels(["", "25%", "50%", "75%", "100%"], fontsize=14)
axs[1].set_ylabel("Performance", fontsize=16)
axs[1].tick_params(axis=u'both', which=u'both',length=0)

axs[1].set_xticks([])
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

fig.savefig("figure1.pdf", bbox_inches='tight')

In [ ]:
download_root = "."

def get_sweep_regression_df_all(sweep_id, allow_crash=False):
    api = wandb.Api()
    sweep = api.sweep("ngruver/physics-uncertainty-exps/{}".format(sweep_id))
    
    results = []
    for run in sweep.runs:        
        config = pd.Series(run.config)
        
        if not allow_crash and not "finished" in str(run):
            continue
        
        if "finished" in str(run):
            summary = pd.Series(run.summary)
        else:
            history = run.history()
            summary = pd.Series({k: history[k].to_numpy()[-1] for k,v in history.items()})
        results.append(pd.concat([config,summary]))
    return pd.concat(results,axis=1).T


sweep_id = "v96kirjy"
df = get_sweep_regression_df_all(sweep_id, allow_crash=True)
sweep_id = "pexiwka8"
df_2 = get_sweep_regression_df_all(sweep_id, allow_crash=True)
full_df = pd.concat([df, df_2])

In [ ]:
df = full_df[full_df["hidden_size"].isin([256])]

metric = "test_gerr"
_df = df[df["num_bodies"].isin([2,3,4])]
gb = _df.groupby(["model_type", "system_type", "num_bodies"])[metric].apply(list).reset_index(name="vals")

model_types = [["NN", "MechanicsNN"],
               ["NonseparableHNN", "HNN"]]
titles = ["NODE Models", "HNN Models"]
groups = ["Without SO bias", "With SO bias"]

colors = ["#6A0078", "#dd63ff"]
sns.set_palette(sns.color_palette("Paired", desat=0.8)[2:4])
matplotlib.rcParams.update({'font.size': 14})

gap_width = 0.4
fig, axs = plt.subplots(1, 2, figsize=(12,3), sharey=True)

for i in range(len(model_types)):
    pts = gb[gb["model_type"].isin(model_types[i])]
    
    num_gaps = (len(pts) // len(model_types[i])) - 1
    num_locs = len(pts) + gap_width * num_gaps
    interval = len(model_types[i]) + gap_width
    
    for j in range(len(model_types[0])):
        locs = np.arange(j, num_locs, interval)[:num_gaps+1]
        _pts = pts[pts["model_type"] == model_types[i][j]]
        model_vals = _pts["vals"]
        system_types = [x.replace('Pendulum','') for x in _pts["system_type"]]
        num_bodies = _pts["num_bodies"]

        means = [np.mean(x) for x in model_vals]
        errs = [np.std(x) / len(x) for x in model_vals]
        
        axs[i].bar(locs, means, width=0.95, yerr=errs, label=groups[j])    
        axs[i].set_yscale('log')
        axs[i].set_yticks([], minor=True)
        if i == 0:
            axs[i].set_ylabel("Rollout Error")

        axs[i].set_xticks(np.arange(0.5, num_locs, interval))
        axs[i].set_xticklabels([f"{system} {n}" for system, n in zip(system_types, num_bodies)])

        axs[i].tick_params(axis=u'both', which=u'both',length=0)
        
        axs[i].set_title(titles[i], fontsize=20, pad=10)
    
plt.tight_layout()
    
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=3)
fig.subplots_adjust(bottom=0.27)

plt.show()

fig.savefig("figure3.pdf", bbox_inches='tight')

In [ ]:
df = full_df[full_df["hidden_size"].isin([256])]

metric = "test_MSE"
_df = df[df["num_bodies"].isin([1,2,3,4])]
gb = _df.groupby(["model_type", "system_type", "num_bodies"])[metric].apply(list).reset_index(name="vals")

model_types = [["MechanicsNN", "HNN"],
               ["MechanicsNN", "HNN"]]
systems = ["ChainPendulum", "SpringPendulum"]
titles = ["Complex Coordinates", "Simple Coordinates"]
groups = ["NODE + SO", "HNN"]

colors = ["#00058A", "#6A0078"]
sns.set_palette(sns.color_palette(colors))
matplotlib.rcParams.update({'font.size': 14})

gap_width = 1.0
fig, axs = plt.subplots(1, 2, figsize=(12,3))

for i in range(len(model_types)):
    pts = gb[(gb["model_type"].isin(model_types[i])) & (gb["system_type"] == systems[i])]
    
    num_gaps = (len(pts) // len(model_types[i])) - 1
    num_locs = len(pts) + gap_width * num_gaps
    interval = len(model_types[i]) + gap_width
    
    for j in range(len(model_types[0])):
        locs = np.arange(j, num_locs, interval)[:num_gaps+1]
        _pts = pts[pts["model_type"] == model_types[i][j]]
        model_vals = _pts["vals"]
        system_types = [x.replace('Pendulum','') for x in _pts["system_type"]]
        num_bodies = _pts["num_bodies"]

        means = [np.mean(x) for x in model_vals]
        errs = [np.std(x) / len(x) for x in model_vals]
        
        axs[i].bar(locs, means, width=0.95, yerr=errs, label=groups[j])    
        axs[i].set_yscale('log')
        axs[i].set_yticks([], minor=True)
        axs[i].set_ylabel("Rollout Error")

        axs[i].set_xticks(np.arange(0.5, num_locs, interval))
        axs[i].set_xticklabels([f"{system} {n}" for system, n in zip(system_types, num_bodies)])

        axs[i].tick_params(axis=u'both', which=u'both',length=0)
        
        axs[i].set_title(titles[i], fontsize=20, pad=10)
    
plt.tight_layout()
    
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=3)
fig.subplots_adjust(bottom=0.27)

plt.show()

fig.savefig("figure4.pdf", bbox_inches='tight')

In [ ]:
sweep_id = "vp3sn5gg"
friction_df = get_sweep_regression_df_all(sweep_id, allow_crash=True)

In [ ]:
df = friction_df

metric = "test_gerr"
_df = df[df["num_bodies"].isin([1,2,3,4])]
gb = _df.groupby(["model_type", "system_type", "num_bodies"])[metric].apply(list).reset_index(name="vals")

model_types = ["NN", "MechanicsNN", "HNN", "MixtureHNN"]
reader_friendly_dict = {
    "NN": "NODE", 
    "MechanicsNN": "NODE + SO",
    "HNN": "HNN",
    "MixtureHNN": "SymODEN"
}

colors = ["#00abdf", "#00058A", "#6A0078", (96/255,74/255,123/255), "#8E6100"]
sns.set_palette(sns.color_palette(colors))

gap_width = 1.
fig, ax = plt.subplots(1, 1, figsize=(10,3))

pts = gb[gb["model_type"].isin(model_types)]

num_gaps = (len(pts) // len(model_types)) - 1
num_locs = len(pts) + gap_width * num_gaps
interval = len(model_types) + gap_width

for i in range(len(model_types)):
    locs = np.arange(i, num_locs, interval)[:num_gaps+1]
    _pts = pts[pts["model_type"] == model_types[i]]

    model_vals = _pts["vals"]
    system_types = [x.replace('Pendulum','').replace('Friction','') for x in _pts["system_type"]]
    num_bodies = _pts["num_bodies"]

    means = [np.mean([v for v in x if not np.isnan(v)]) for x in model_vals]
    errs = [np.std([v for v in x if not np.isnan(v)]) / len(x) for x in model_vals]

    ax.bar(locs, means, width=0.95, yerr=errs, label=reader_friendly_dict[model_types[i]])    

ax.set_yscale('log')
ax.set_yticks([], minor=True)
ax.set_ylabel("Rollout Error")

ax.set_xticks(np.arange(1.5, num_locs, interval))
ax.set_xticklabels([f"{system} {n}" for system, n in zip(system_types, num_bodies)])

ax.tick_params(axis=u'both', which=u'both',length=0)

ax.set_xlim(-1, num_locs)
# ax.set_title(titles[i])
    
plt.tight_layout()
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4)
fig.subplots_adjust(bottom=0.27)

plt.show()

fig.savefig("figure5.pdf", bbox_inches='tight')

In [ ]:
def get_sweep_regression_df_all(sweep_id, allow_crash=False):
    api = wandb.Api()
    sweep = api.sweep("samuelstanton/physics-uncertainty-exps/{}".format(sweep_id))
    
    results = []
    for run in sweep.runs:        
        config = pd.Series(run.config)
        
        if not allow_crash and not "finished" in str(run):
            continue
        
        if "finished" in str(run):
            summary = pd.Series(run.summary)
        else:
            history = run.history()
            summary = pd.Series({k: history[k].to_numpy()[-1] for k,v in history.items()})
#         print(summary)
        results.append(pd.concat([config,summary]))
    return pd.concat(results,axis=1).T


sweep_id = "wkn77qza"
df = get_sweep_regression_df_all(sweep_id, allow_crash=True)

In [ ]:
sweep_id = "ahybaa2m"
df_2 = get_sweep_regression_df_all(sweep_id, allow_crash=True)
df = pd.concat([df, df_2])

In [ ]:
sweep_id = "glupfh56"
df_3 = get_sweep_regression_df_all(sweep_id, allow_crash=True)
df = pd.concat([df, df_3])

In [ ]:
df = df[(df["hidden_size"] == 128) & (df["subsample_ratio"] == 1.)]
df["task"] = df["task"].apply(lambda s: s.replace("Full-v0", ""))

reader_friendly_dict = {
    "NODE": "NODE",
    "CoupledNODE": "NODE + SO",
    "MixtureHNN": "SymODEN"
}

def bar_plot(ax, title, df, metric, group_label, methods, ind_label, keys=None):
    if keys is None:
        keys = df[ind_label].drop_duplicates().to_numpy()
        keys = keys[np.argsort(keys)]
    
    for idx, method in enumerate(methods):
        _df = df[df[group_label] == method]
        locs = idx + (len(methods) + 1) * np.arange(len(keys))
        means, stds = [],[]
        for idx, k in enumerate(keys):
            try:
                __df = _df[_df[ind_label] == k]
                __df = __df[~__df[metric].isna()]
                if title == "Test":
                    data = [x for x in __df[metric].to_numpy()]
                else:
                    data = [x for x in __df[metric].to_numpy()]
                means.append(np.mean(data))
                stds.append(np.std(data))
            except Exception as e:
                print(e)
                locs = np.delete(locs, idx)
                continue
        label = method if method != "FlexHNN" else "RPP"
        ax.bar(locs, means, yerr=stds, label=reader_friendly_dict[method], width=0.9)
    
    ax.set_yscale('log')
    ax.set_xticks(np.arange(1, len(keys) * len(methods) + len(keys) - 1, len(methods) + 1))
    ax.set_xticklabels(keys)#, rotation = 20)
    ax.set_ylabel("Rollout Err")
    ax.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_title(title, fontsize=22, pad=10) 

colors = ["#00abdf", "#00058A", (96/255,74/255,123/255), "#8E6100"]
sns.set_palette(sns.color_palette(colors))
matplotlib.rcParams.update({'font.size': 18})

metric = "test_mse"
methods = ["NODE", "CoupledNODE", "MixtureHNN"]

fig, ax = plt.subplots(1, 2, figsize=(12,4))
bar_plot(ax[0], "Test", df, "test_mse", "model_type", methods, "task")
bar_plot(ax[1], "Train", df, "train_mse", "model_type", methods, "task")

plt.tight_layout()

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, ncol=3, loc='lower center')
fig.subplots_adjust(bottom=0.25)#, left=-0.4)

plt.show()

fig.savefig("figure7.pdf", bbox_inches='tight')

In [ ]:
def get_sweep_regression_df_all(sweep_id, allow_crash=False):
    api = wandb.Api()
    sweep = api.sweep("ngruver/physics-uncertainty-exps/{}".format(sweep_id))
    
    results = []
    for run in sweep.runs:        
        config = pd.Series(run.config)
        
        if not allow_crash and not "finished" in str(run):
            continue
        
        if "finished" in str(run):
            summary = pd.Series(run.summary)
        else:
            history = run.history()
            summary = pd.Series({k: history[k].to_numpy()[-1] for k,v in history.items()})
        results.append(pd.concat([config,summary]))
    return pd.concat(results,axis=1).T

sweep_id = "9f0ldbr6"
df1 = get_sweep_regression_df_all(sweep_id, allow_crash=True)
sweep_id = "p400ew2f"
df2 = get_sweep_regression_df_all(sweep_id, allow_crash=True)
df = pd.concat([df1, df2])

In [ ]:
metric = "test_gerr"
_df = df[df["num_bodies"].isin([1,2,3,4])]
gb = _df.groupby(["model_type", "system_type", "loss", "hidden_size"])[metric].apply(list).reset_index(name="vals")
gb["mean"] = pd.DataFrame(gb["vals"].values.tolist()).mean(1)
gb

In [ ]:
gb = gb[gb["hidden_size"] == 256]

In [ ]:
model_types = ["NN", "MechanicsNN", "HNN"]
reader_friendly_dict = {
    "NN": "NODE", 
    "MechanicsNN": "NODE + SO",
    "HNN": "HNN",
}

colors = ["#00abdf", "#00058A", "#6A0078", (96/255,74/255,123/255), "#8E6100"]
sns.set_palette(sns.color_palette(colors))
matplotlib.rcParams.update({'font.size': 18})

gap_width = 1.
fig, ax = plt.subplots(1, 1, figsize=(10,3))

pts = gb[gb["model_type"].isin(model_types)]

num_gaps = (len(pts) // len(model_types)) - 1
num_locs = len(pts) + gap_width * num_gaps
interval = len(model_types) + gap_width

for i in range(len(model_types)):
    locs = np.arange(i, num_locs, interval)[:num_gaps+1]
    _pts = pts[pts["model_type"] == model_types[i]]

    model_vals = _pts["vals"]
    system_types = [x.replace('Pendulum','').replace('Friction','') for x in _pts["system_type"]]
    num_bodies = _pts["loss"]
    hidden_size = _pts["hidden_size"]

    means = [np.mean([v for v in x if not np.isnan(v)]) for x in model_vals]
    errs = [np.std([v for v in x if not np.isnan(v)]) / len(x) for x in model_vals]

    ax.bar(locs, means, width=0.95, yerr=errs, label=reader_friendly_dict[model_types[i]])    

ax.set_yscale('log')
ax.set_yticks([], minor=True)
ax.set_ylabel("Rollout Error")

ax.set_xticks(np.arange(1.5, num_locs, interval))
ax.set_xticklabels([f"{system} ({n} loss)" for system, n in zip(system_types, num_bodies)])
ax.tick_params(axis=u'both', which=u'both',length=0)

ax.set_xlim(-1, num_locs)
 
plt.tight_layout()
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4)
fig.subplots_adjust(bottom=0.35)

plt.show()

fig.savefig("l1-l2-loss-comparison.pdf", bbox_inches='tight')

In [ ]:
import tqdm 

def get_sweep_regression_df_all(sweep_id, allow_crash=False):
    api = wandb.Api(timeout=60)
    sweep = api.sweep("ngruver/physics-uncertainty-exps/{}".format(sweep_id))
    
    results = []
    for run in tqdm.tqdm(sweep.runs):        
        config = pd.Series(run.config)
        
        if not allow_crash and not "finished" in str(run):
            continue
             
        history = run.history()
        summary = pd.Series({k: min(history[k].to_numpy()) for k,v in history.items()})
        results.append(pd.concat([config,summary]))
    return pd.concat(results,axis=1).T

sweep_id = "s9fjv7t1" #"/tpijuz7e"
df = get_sweep_regression_df_all(sweep_id, allow_crash=True)

In [ ]:
df = _df[_df["n_systems"] == 5000]
metric = "test_gerr"
gb = df.groupby(["model_type", "system_type", "n_systems", "data_seed"])[metric].apply(list).reset_index(name="vals")
gb["min_val"] = pd.DataFrame(gb["vals"].values.tolist()).min(1)
gb = gb.drop(columns=['vals'])
gb = gb.groupby(["model_type", "system_type", "n_systems"])["min_val"].apply(list).reset_index(name="vals")
gb["mean"] = pd.DataFrame(gb["vals"].values.tolist()).mean(1)

In [ ]:
model_types = ["NN", "MechanicsNN", "HNN"]
reader_friendly_dict = {
    "NN": "NODE", 
    "MechanicsNN": "NODE + SO",
    "HNN": "HNN",
}

colors = ["#00abdf", "#00058A", "#6A0078", (96/255,74/255,123/255), "#8E6100"]
sns.set_palette(sns.color_palette(colors))
matplotlib.rcParams.update({'font.size': 18})

gap_width = 1.
fig, ax = plt.subplots(1, 1, figsize=(10,3))

pts = gb[gb["model_type"].isin(model_types)]

num_gaps = (len(pts) // len(model_types)) - 1
num_locs = len(pts) + gap_width * num_gaps
interval = len(model_types) + gap_width

for i in range(len(model_types)):
    locs = np.arange(i, num_locs, interval)[:num_gaps+1]
    _pts = pts[pts["model_type"] == model_types[i]]

    print(_pts.drop(columns=['vals']))
    model_vals = _pts["vals"]
    system_types = [x.replace('Pendulum','').replace('Friction','') for x in _pts["system_type"]]
    n_systems = _pts["n_systems"]
    
    means = [np.mean([v for v in x if not np.isnan(v)]) for x in model_vals]
    errs = [np.std([v for v in x if not np.isnan(v)]) / len(x) for x in model_vals]

    ax.bar(locs, means, width=0.95, yerr=errs, label=reader_friendly_dict[model_types[i]])    

ax.set_yscale('log')
ax.set_yticks([], minor=True)
ax.set_ylabel("Rollout Error")

ax.set_xticks(np.arange(1.5, num_locs, interval))
ax.set_xticklabels([f"{system}" for n, system in zip(n_systems, system_types)])

ax.tick_params(axis=u'both', which=u'both',length=0)

ax.set_xlim(-1, num_locs)
   
plt.tight_layout()
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4)
fig.subplots_adjust(bottom=0.35)

plt.show()

fig.savefig("additional_systems.pdf", bbox_inches='tight')

In [ ]:
from matplotlib import ticker

colors = ["#00abdf", "#00058A", "#6A0078", (96/255,74/255,123/255), "#8E6100"]
sns.set_palette(sns.color_palette(colors))
matplotlib.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(1, 1, figsize=(3,3))

ax.bar(0, 89.8190, width=0.95, label="NODE")    
ax.bar(1, 85.6026, width=0.95, label="NODE + SO") 
ax.bar(2, 85.1126, width=0.95, label="HNN") 

ax.set_yscale('log')
ax.set_yticks([86, 88, 90], minor=True)
ax.yaxis.set_minor_formatter(ticker.ScalarFormatter())
ax.set_ylabel("Rollout Error")

ax.tick_params(axis=u'both', which=u'both',length=0)

ax.set_xlim(-1, 3)
ax.set_xticks([1])
ax.set_xticklabels(["Pendulum w/ Contact"])
    
plt.tight_layout()
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=3)
fig.subplots_adjust(bottom=0.35)

plt.show()

fig.savefig("contact_system.pdf", bbox_inches='tight')